In [54]:
#!pip install folium

In [55]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [56]:
List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [57]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [58]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [59]:
df=df[df['Borough']!='Not assigned']

In [60]:
df['Borough']=df[df['Neighborhood']=='Not assigned']

In [61]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,NaN,Parkwoods
3,M4A,NaN,Victoria Village
4,M5A,NaN,"Regent Park, Harbourfront"
5,M6A,NaN,"Lawrence Manor, Lawrence Heights"
6,M7A,NaN,"Queen's Park, Ontario Provincial Government"


In [62]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [63]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [64]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [65]:
df_merge.drop_duplicates(inplace=True)

In [66]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [67]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,NaN,Parkwoods
1,M4A,NaN,Victoria Village
2,M5A,NaN,"Regent Park, Harbourfront"
3,M6A,NaN,"Lawrence Manor, Lawrence Heights"
4,M7A,NaN,"Queen's Park, Ontario Provincial Government"


In [68]:
df_merge.shape

(103, 3)

In [69]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [70]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [71]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [72]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [73]:
geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,NaN,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,NaN,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,NaN,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,NaN,Woburn
4,M1H,43.773136,-79.239476,NaN,Cedarbrae


In [75]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,NaN,"Malvern, Rouge",43.806686,-79.194353
1,M1C,NaN,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,NaN,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,NaN,Woburn,43.770992,-79.216917
4,M1H,NaN,Cedarbrae,43.773136,-79.239476


In [76]:
geo_data["Borough"].fillna('empty', inplace=True)
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,empty,"Malvern, Rouge",43.806686,-79.194353
1,M1C,empty,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,empty,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,empty,Woburn,43.770992,-79.216917
4,M1H,empty,Cedarbrae,43.773136,-79.239476


In [84]:
toronto_data=geo_data[geo_data['Neighborhood'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
40,M4J,empty,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106
46,M4R,empty,"North Toronto West, Lawrence Park",43.715383,-79.405678
59,M5J,empty,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
60,M5K,empty,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
66,M5S,empty,"University of Toronto, Harbord",43.662696,-79.400049


In [85]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180604'

In [86]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

East Toronto, Broadview North (Old East York)
North Toronto West,  Lawrence Park
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
University of Toronto, Harbord
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
New Toronto, Mimico South, Humber Bay Shores


In [88]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,The Path,43.683923,-79.335007,Park
1,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store
2,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,Donlands Subway Station,43.680960,-79.337759,Metro Station
3,"North Toronto West, Lawrence Park",43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
4,"North Toronto West, Lawrence Park",43.715383,-79.405678,Starbucks,43.715590,-79.400450,Coffee Shop


In [89]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"East Toronto, Broadview North (Old East York)",3,3,3,3,3,3
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100
"New Toronto, Mimico South, Humber Bay Shores",13,13,13,13,13,13
"North Toronto West, Lawrence Park",18,18,18,18,18,18
"Toronto Dominion Centre, Design Exchange",100,100,100,100,100,100
"University of Toronto, Harbord",36,36,36,36,36,36


In [90]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(285, 108)

In [91]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Baseball Stadium,...,Sushi Restaurant,Taco Place,Tailor Shop,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,"Business reply mail Processing Centre, South C...",0.000000,0.00,0.00,0.0,0.066667,0.000000,0.0,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.066667
1,"East Toronto, Broadview North (Old East York)",0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.000000
2,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.05,0.01,0.0,0.000000,0.010000,0.0,0.02,0.02,...,0.01,0.0,0.0,0.01,0.01,0.01,0.01,0.0,0.01,0.000000
3,"New Toronto, Mimico South, Humber Bay Shores",0.076923,0.00,0.00,0.0,0.000000,0.076923,0.0,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.000000
4,"North Toronto West, Lawrence Park",0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.055556


In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Business reply mail Processing Centre, South C...",Yoga Studio,Brewery,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Restaurant
1,"East Toronto, Broadview North (Old East York)",Convenience Store,Park,Metro Station,Food Court,Convention Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
2,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Restaurant,Italian Restaurant,Sporting Goods Shop,Scenic Lookout
3,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Café,Fried Chicken Joint,Liquor Store,Fast Food Restaurant,Mexican Restaurant,Pharmacy,Pizza Place,Restaurant,Coffee Shop
4,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Spa,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Rental Car Location,Restaurant,Chinese Restaurant


###### Making Clusters for Neighborhood

In [94]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 2, 4, 0, 2, 2], dtype=int32)

In [95]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,M4J,empty,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,1,Convenience Store,Park,Metro Station,Food Court,Convention Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
46,M4R,empty,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Clothing Store,Spa,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Rental Car Location,Restaurant,Chinese Restaurant
59,M5J,empty,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2,Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Restaurant,Italian Restaurant,Sporting Goods Shop,Scenic Lookout
60,M5K,empty,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Italian Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,Tea Room
66,M5S,empty,"University of Toronto, Harbord",43.662696,-79.400049,2,Café,Bookstore,Italian Restaurant,Japanese Restaurant,Theater,Restaurant,Bakery,Bar,College Gym,Chinese Restaurant


In [96]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,"Business reply mail Processing Centre, South C...",Yoga Studio,Brewery,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Restaurant
1,1,"East Toronto, Broadview North (Old East York)",Convenience Store,Park,Metro Station,Food Court,Convention Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
2,2,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Restaurant,Italian Restaurant,Sporting Goods Shop,Scenic Lookout
3,4,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Café,Fried Chicken Joint,Liquor Store,Fast Food Restaurant,Mexican Restaurant,Pharmacy,Pizza Place,Restaurant,Coffee Shop
4,0,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Spa,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Rental Car Location,Restaurant,Chinese Restaurant


In [97]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


###### Creating Map

In [98]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters